# Setup

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import statistics as stat
from IPython.display import display

class par: None
class moms: None
    
# from farhi and gourio
class true: None
true.beta = 0.961
true.mu = 1.079
true.delta = 0.02778
true.alpha = 0.244
true.g_Z = 0.01298
true.r_star = np.nan
true.g_T = np.nan
true.p = 0.034

## Settings and assumptions

In [2]:
# a. shock size
par.b = -np.log(1-0.15)

# b. risk aversion coefficient
par.theta = 12

# c. IES, sigma = 1/IES
par.sigma = 0.5

# Load data

In [3]:
df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
df.info()
df = df[['year','ProfitK','ProfitY','RF', 'growthpop','priceinvt','PD','growthTFP','ik','EmpPop']]
df = df.set_index('year')
display(df.head())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 42 columns):
year            57 non-null int64
ProfitK         57 non-null float64
ProfitY         57 non-null float64
IY              57 non-null float64
RF              57 non-null float64
PE              57 non-null float64
growthpop       57 non-null float64
growthemp       57 non-null float64
growthGDP       57 non-null float64
priceinvt       57 non-null float64
PD              57 non-null float64
PEop            57 non-null float64
growthTFP       57 non-null float64
growthTFPADJ    57 non-null float64
ERP             57 non-null float64
KY              57 non-null float64
ystar           57 non-null float64
Drate           57 non-null float64
ik              57 non-null float64
LS              57 non-null float64
CS              57 non-null float64
PS              57 non-null float64
SP              57 non-null float64
ERPalt          57 non-null float64
SPMU            57 non-null float64

,ProfitK,ProfitY,RF,growthpop,priceinvt,PD,growthTFP,ik,EmpPop
year,,,,,,,,,
1960,14.234645,29.143753,2.101667,1.646313,-1.321805,28.767187,0.647897,7.456101,56.108616
1961,14.418177,29.418518,1.855833,1.292520,-1.373270,33.813931,1.891988,7.261791,55.354179
1962,15.298585,30.143707,1.725833,1.157498,-1.162756,29.106068,3.314460,7.622490,55.513981
1963,16.091473,31.001640,2.081667,1.866188,-1.561601,30.914055,2.858746,7.922162,55.351719
1964,16.331043,31.335564,2.345833,1.675677,-0.907530,32.665569,2.761517,8.154409,55.670570


,ProfitK,ProfitY,RF,growthpop,priceinvt,PD,growthTFP,ik,EmpPop
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000
mean,14.346346,31.046356,1.414357,1.381947,-1.001414,38.333945,1.033962,7.749363,59.781546
std,1.269753,2.538903,1.995644,0.438917,1.296615,15.465343,1.618604,0.693920,2.719538
min,11.476599,27.352089,-2.533333,0.749057,-4.067451,18.496330,-3.801344,5.915066,55.351719
25%,13.559713,29.275795,-0.170000,1.022081,-1.778501,27.761936,0.119928,7.295365,57.774712
50%,14.230997,30.411072,1.451667,1.236426,-1.073029,33.334805,1.153324,7.883199,59.327110
75%,15.284401,31.860176,2.786667,1.675677,-0.097202,46.440563,2.176149,8.207948,62.339596
max,17.108320,36.831299,5.945833,2.749740,3.547138,89.555916,3.372487,9.171547,64.400581


# Moments

In [4]:
moms.ProfitK = stat.mean(df.loc[1984:2000,"ProfitK"])/100
moms.s_K = stat.mean(df.loc[1984:2000,"ProfitY"])/100
moms.ik = stat.mean(df.loc[1984:2000,"ik"])/100
moms.PD = stat.mean(df.loc[1984:2000,"PD"])
moms.TFPgrowth = stat.mean(df.loc[1984:2000,"growthTFP"])/100
moms.RF = stat.mean(df.loc[1984:2000,"RF"])/100

for name in ['ProfitK','s_K','ik','PD','TFPgrowth']:
    print(f'{name:10s} = {getattr(moms,name):.5f}')

ProfitK    = 0.14012
s_K        = 0.29887
ik         = 0.08103
PD         = 42.33588
TFPgrowth  = 0.01100


# Step 1

Estimate $g_L$, $g_Q$ and $\bar{N}$ directly from data

In [5]:
par.g_L = stat.mean(df.loc[1984:2000,"growthpop"])/100
par.g_Q = -stat.mean(df.loc[1984:2000,"priceinvt"])/100 # note: negativ value used
par.N_bar = stat.mean(df.loc[1984:2000,"EmpPop"])/100

for name in ['g_L','g_Q','N_bar']:
    print(f'{name:5s} = {getattr(par,name):.5f}')

g_L   = 0.01171
g_Q   = 0.01769
N_bar = 0.62344


# Step 2

Estimate $\mu$, $\alpha$, $\delta$, $g_z$ by solving equation system

In [6]:
def eq_footnote_15(par,moms):
     return moms.TFPgrowth - (par.g_T-(1-moms.s_K)*par.g_L-moms.s_K*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_T) - (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha))

def eq_15(par,moms):

    qstar = 1 # normalization
    beta_star = 1/(1+par.r_star)
    
    temp = (1+par.g_Q)/beta_star-(1-par.delta)
    kstar = (temp*par.mu/(par.alpha*qstar))**(1/(par.alpha-1))*par.N_bar
    ystar = kstar**par.alpha*par.N_bar**(1-par.alpha)
    pistar = (par.mu+par.alpha-1)/par.mu*ystar

    return moms.ProfitK-pistar/(kstar/qstar) 
    
def eq_18(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta) - moms.ik

def eq_20(par,moms):
    return moms.s_K - (par.mu+par.alpha-1)/(par.mu)

def eq_23(par,moms):
    beta_star = 1/(1+par.r_star)
    return moms.PD - beta_star*(1+par.g_T)/(1-beta_star*(1+par.g_T))

#def eq_27(par,moms):
#    return moms.ProfitK - (par.mu+par.alpha-1)/par.alpha*(par.r_star+par.delta+par.g_Q)

In [7]:
def set_parameters(par,x):
    
    for name,value in zip(par.names,x):
        setattr(par,name,value)
    
def set_x(par):
    
    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)
        
    return x

def eq_system(x,par,moms):
    
    # a. set parameters
    set_parameters(par,x)            
    
    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_15(par,moms))          
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))
    out.append(eq_23(par,moms))

    return out

In [8]:
# a. parameters to estimate
par.names = ['g_Z','g_T','delta','alpha','r_star','mu']

# b. guess
par.mu = 1.01
par.delta = 0.025
par.alpha = 0.25
par.g_Z = 0.02
par.r_star = 0.05
par.g_T = 0.04
x = set_x(par)

# c. solve
solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
set_parameters(par,solution)

# d. print result
for name in par.names:
    print(f'{name:10} = {getattr(par,name):.8f} [FH: {getattr(true,name):.4f}]')        

g_Z        = 0.01298405 [FH: 0.0130]
g_T        = 0.03494270 [FH: nan]
delta      = 0.02777669 [FH: 0.0278]
alpha      = 0.24368407 [FH: 0.2440]
r_star     = 0.05938870 [FH: nan]
mu         = 1.07871675 [FH: 1.0790]


# Step 3

Estimate $\beta$ and $p$

In [9]:
def update_misc(par):
    
    par.beta_star = 1/(1+par.r_star)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_PC = (1+par.g_T)/(1+par.g_L)-1
    

In [10]:
def find_p(p,par,moms):
        
    update_misc(par)        
    MOM2 = ((1-2*p)+p*np.exp(-par.bh*(1-par.theta)) + p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*p)+p*np.exp(-par.bh*(-par.theta)) + p*np.exp(-par.b*(-par.theta))) 
    
    return moms.RF - (MOM2/(par.beta_star*MOM3)-1)

**Find p:**

In [11]:
par.p = optimize.fsolve(find_p, 0.02, args=(par,moms), full_output=0)[0]
update_misc(par)

**Find $\beta$:**

In [12]:
MOM =  ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
par.beta = par.beta_star/((1+par.g_PC)**(-par.sigma)*MOM);

In [13]:
for name in ['beta','p']:
   print(f'{name:10} = {getattr(par,name):4f} [FH: {getattr(true,name):4f}]')

beta       = 0.960540 [FH: 0.961000]
p          = 0.034146 [FH: 0.034000]
